In [23]:
import uuid
from db_utils import DatabaseManager
from text_chunking_util import SimpleTextChunkCreator
from models.db_models import AudioChunk


def saveChunksForJob(db_manager: DatabaseManager, job_id: str):
    """
    Create audio chunks from job's firstDraft and save to database
    
    Args:
        db_manager: Database manager instance
        job_id: ID of the job to process
    """
    # Fetch job
    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    
    # Check if audioChunks already exist (idempotent behavior)
    if job.audioChunks and len(job.audioChunks) > 0:
        print(f"✓ Audio chunks already exist for job {job_id}")
        print(f"  Existing chunks: {len(job.audioChunks)}")
        return job.audioChunks
    
    # Get firstDraft text
    first_draft = job.firstDraft
    if not first_draft:
        raise ValueError(f"Job {job_id} has no firstDraft content")
    
    # Create chunks using SimpleTextChunkCreator
    chunker = SimpleTextChunkCreator()
    text_chunks = chunker.convertTextToChunks(first_draft, '\n')
    
    # Convert to AudioChunk objects with unique IDs
    audio_chunks = []
    for index, chunk_text in enumerate(text_chunks):
        chunk_id = str(uuid.uuid4())[:8]  # Short unique ID
        
        audio_chunk = AudioChunk(
            chunkId=chunk_id,
            text=chunk_text,
            outputAudioFilePath=""  # Will be populated during audio generation
        )
        audio_chunks.append(audio_chunk)
    
    # Update job with audio chunks
    job.audioChunks = audio_chunks
    db_manager.update_job_field(job_id, job)
    
    print(f"✓ Created {len(audio_chunks)} audio chunks for job {job_id}")
    return audio_chunks

In [24]:
import re
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from tts_util import OpenAiTTSProvider
from typing import Tuple, List
from models.db_models import AudioChunk


def convertTextChunksToAudio(job_id: str, db_manager: DatabaseManager) -> Tuple[List[AudioChunk], str]:
    """
    Convert text chunks to audio files using parallel processing
    
    Args:
        job_id: ID of the job to process
        db_manager: Database manager instance
        
    Returns:
        Tuple[List[AudioChunk], str]: (audio_chunks, folder_path)
    """
    # Fetch job
    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    
    # Check if audioChunks array is populated
    if not job.audioChunks or len(job.audioChunks) == 0:
        raise ValueError(f"Job {job_id} has no audio chunks to process")
    
    # Create folder name from first chunk's text (always create for consistency)
    first_chunk_text = job.audioChunks[0].text[:50]  # Limit to first 50 chars
    # Clean text for folder name (remove special chars, keep alphanumeric and spaces)
    clean_text = re.sub(r'[^\w\s-]', '', first_chunk_text).strip()
    clean_text = re.sub(r'[-\s]+', '_', clean_text)  # Replace spaces and hyphens with underscore
    
    # Create folder with YYYYMMDD_text format
    date_str = datetime.now().strftime('%Y%m%d')
    folder_name = f"{date_str}_{clean_text}"
    folder_path = f"./audio_outputs/audio_chunks/{folder_name}"
    
    # Identify chunks that need audio generation
    chunks_to_process = []
    completed_count = 0
    
    for i, chunk in enumerate(job.audioChunks):
        if chunk.outputAudioFilePath and chunk.outputAudioFilePath.strip():
            completed_count += 1
        else:
            chunks_to_process.append((i, chunk))
    
    print(f"Audio generation status for job {job_id}:")
    print(f"  Already completed: {completed_count}")
    print(f"  Needs processing: {len(chunks_to_process)}")
    print(f"  Total chunks: {len(job.audioChunks)}")
    
    # If all chunks are already completed, return early
    if len(chunks_to_process) == 0:
        print(f"✓ All audio chunks already generated for job {job_id}")
        return job.audioChunks, folder_path
    
    print(f"Creating audio files in folder: {folder_path}")
    
    # Initialize TTS provider
    tts_provider = OpenAiTTSProvider(max_retries=3, retry_delay=1.0)
    
    def process_chunk(index, audio_chunk):
        """Process a single audio chunk"""
        try:
            file_path = tts_provider.generate_speech(
                voice="echo",  # Default voice
                instructions="Clear and engaging narration",  # Default instructions
                input_text=audio_chunk.text,
                folder=folder_path,
                chunk_index=index,
                chunk_id=audio_chunk.chunkId
            )
            return index, file_path, None
        except Exception as e:
            return index, None, str(e)
    
    # Process only chunks that need audio generation
    with ThreadPoolExecutor(max_workers=5) as executor:
        # Submit only incomplete chunks
        future_to_index = {
            executor.submit(process_chunk, i, chunk): i 
            for i, chunk in chunks_to_process
        }
        
        processed_count = 0
        failed_count = 0
        
        # Process results as they complete
        for future in as_completed(future_to_index):
            index, file_path, error = future.result()
            
            if error:
                print(f"✗ Failed to process chunk {index}: {error}")
                failed_count += 1
            else:
                # Update the chunk with the file path
                job.audioChunks[index].outputAudioFilePath = file_path
                print(f"✓ Processed chunk {index}: {file_path}")
                processed_count += 1
    
    # Update job in database
    db_manager.update_job_field(job_id, job)
    
    print(f"\n✓ Audio generation completed!")
    print(f"  Successfully processed: {processed_count}")
    print(f"  Failed: {failed_count}")
    print(f"  Previously completed: {completed_count}")
    print(f"  Total chunks: {len(job.audioChunks)}")
    
    return job.audioChunks, folder_path

In [25]:
import os
import subprocess
from typing import List


def stitchAudioChunks(job_id: str, folder_path: str, db_manager: DatabaseManager, gap_seconds: float = 1.0) -> str:
    """
    Stitch audio chunks into a single final audio file using FFmpeg
    
    Args:
        job_id: ID of the job to process
        folder_path: Path to folder containing audio chunk files
        db_manager: Database manager instance
        gap_seconds: Gap in seconds between audio chunks (default: 1.0)
        
    Returns:
        str: Path to the final stitched audio file
    """
    # Fetch job
    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    
    # Check if audioChunks array is populated
    if not job.audioChunks or len(job.audioChunks) == 0:
        raise ValueError(f"Job {job_id} has no audio chunks to stitch")
    
    # Check if all chunks have outputAudioFilePath
    missing_audio_chunks = []
    valid_audio_files = []
    
    for i, chunk in enumerate(job.audioChunks):
        if not chunk.outputAudioFilePath or not chunk.outputAudioFilePath.strip():
            missing_audio_chunks.append(i)
        elif os.path.exists(chunk.outputAudioFilePath):
            valid_audio_files.append(chunk.outputAudioFilePath)
        else:
            print(f"⚠️  Audio file not found: {chunk.outputAudioFilePath}")
    
    if missing_audio_chunks:
        raise ValueError(f"Job {job_id} has chunks without audio files at indices: {missing_audio_chunks}")
    
    if not valid_audio_files:
        raise ValueError(f"No valid audio files found for job {job_id}")
    
    print(f"Stitching {len(valid_audio_files)} audio chunks for job {job_id}")
    print(f"Gap between chunks: {gap_seconds} seconds")
    
    # Create output directory
    output_dir = "./audio_outputs/final_audio"
    os.makedirs(output_dir, exist_ok=True)
    
    # Generate filename from folder path
    folder_name = os.path.basename(folder_path.rstrip('/'))
    final_filename = f"{folder_name}.wav"
    final_file_path = os.path.join(output_dir, final_filename)
    
    try:
        # Create a temporary file list for ffmpeg
        temp_file_list = os.path.join(output_dir, f"temp_filelist_{job_id}.txt")
        
        with open(temp_file_list, 'w') as f:
            for i, audio_file in enumerate(valid_audio_files):
                # Add the audio file
                f.write(f"file '{os.path.abspath(audio_file)}'\n")
                
                # Add silence gap between files (except after the last file)
                if i < len(valid_audio_files) - 1:
                    # Create a temporary silence file for the gap
                    silence_file = os.path.join(output_dir, f"temp_silence_{job_id}.wav")
                    silence_cmd = [
                        'ffmpeg', '-f', 'lavfi', '-i', f'anullsrc=r=44100:cl=stereo', 
                        '-t', str(gap_seconds), '-y', silence_file
                    ]
                    subprocess.run(silence_cmd, capture_output=True, check=True)
                    f.write(f"file '{os.path.abspath(silence_file)}'\n")
        
        # Use ffmpeg to concatenate all files
        ffmpeg_cmd = [
            'ffmpeg', '-f', 'concat', '-safe', '0', '-i', temp_file_list,
            '-c', 'copy', '-y', final_file_path
        ]
        
        print(f"Running FFmpeg to stitch audio files...")
        result = subprocess.run(ffmpeg_cmd, capture_output=True, text=True)
        
        if result.returncode != 0:
            raise Exception(f"FFmpeg failed: {result.stderr}")
        
        # Clean up temporary files
        if os.path.exists(temp_file_list):
            os.remove(temp_file_list)
        
        silence_file = os.path.join(output_dir, f"temp_silence_{job_id}.wav")
        if os.path.exists(silence_file):
            os.remove(silence_file)
        
        # Verify the output file was created
        if not os.path.exists(final_file_path):
            raise Exception(f"Output file was not created: {final_file_path}")
        
        # Update job with final audio path
        job.finalAudioFilePath = final_file_path
        db_manager.update_job_field(job_id, job)
        
        # Get file size for reporting
        file_size = os.path.getsize(final_file_path)
        
        print(f"\n✓ Audio stitching completed!")
        print(f"  Processed chunks: {len(valid_audio_files)}/{len(job.audioChunks)}")
        print(f"  Final audio file size: {file_size / 1024 / 1024:.1f} MB")
        print(f"  Final audio saved to: {final_file_path}")
        
        return final_file_path
        
    except subprocess.CalledProcessError as e:
        raise Exception(f"FFmpeg command failed: {e}")
    except Exception as e:
        raise Exception(f"Audio stitching failed: {e}")
    finally:
        # Cleanup temporary files in case of error
        temp_file_list = os.path.join(output_dir, f"temp_filelist_{job_id}.txt")
        if os.path.exists(temp_file_list):
            os.remove(temp_file_list)
        
        silence_file = os.path.join(output_dir, f"temp_silence_{job_id}.wav")
        if os.path.exists(silence_file):
            os.remove(silence_file)

In [18]:
# Test tts_util.py
from tts_util import OpenAiTTSProvider

ttsProvider = OpenAiTTSProvider()
print(ttsProvider.generate_speech('echo', 'Calm and soothing', 'Hello, world! How is it going. I am Abhilash. This is a test audio. I am an entrepreneure. I like to build cool things. You mother fucker', './audio_outputs', 1, 'test'))

INFO:tts_util:Generating audio for chunk 1_test, attempt 1
INFO:tts_util:Successfully generated audio: ./audio_outputs/001_test.wav


./audio_outputs/001_test.wav


In [19]:
# Test chunk creation util
from text_chunking_util import SimpleTextChunkCreator
chunkCreator = SimpleTextChunkCreator()
text = """Hey there. I am testing chunking logic.It should chunk based on new line characters."""
chunks = chunkCreator.convertTextToChunks(text, "\n")
print(chunks)

['Hey there. I am testing chunking logic.It should chunk based on new line characters.']


In [ ]:
import os
from dotenv import load_dotenv
from db_utils import DatabaseManager

if __name__ == "__main__":
    load_dotenv()
    
    # Initialize database manager
    db_manager = DatabaseManager()
    
    # Set the job ID (replace with actual job ID from story generation)
    job_id = "68e0c9e2356de3c49a902ead"
    
    # Step 1: Create and save audio chunks for the job
    print("=== STEP 1: Creating Audio Chunks ===")
    audio_chunks = saveChunksForJob(db_manager, job_id)
    
    print(f"Processed {len(audio_chunks)} chunks:")
    for i, chunk in enumerate(audio_chunks):
        print(f"  Chunk {i+1}: ID={chunk.chunkId}, Length={len(chunk.text)} chars")
    
    # Step 2: Convert text chunks to audio files
    print("\n=== STEP 2: Converting Chunks to Audio ===")
    processed_chunks, folder_path = convertTextChunksToAudio(job_id, db_manager)
    
    print(f"\n✓ Audio pipeline completed!")
    print(f"  Audio files saved to: {folder_path}")
    print(f"  Total processed chunks: {len(processed_chunks)}")
    
    # Step 3: Stitch audio chunks into final audio file
    print("\n=== STEP 3: Stitching Audio Chunks ===")
    final_audio_path = stitchAudioChunks(job_id, folder_path, db_manager, gap_seconds= 0.1)
    
    print(f"\n🎉 Complete audio generation pipeline finished!")
    print(f"  Final audio file: {final_audio_path}")
    
    # Close DB connection when done
    db_manager.close()

INFO:tts_util:Generating audio for chunk 4_2332ad90, attempt 1
INFO:tts_util:Generating audio for chunk 1_ee238105, attempt 1
INFO:tts_util:Generating audio for chunk 0_0f6671d4, attempt 1
INFO:tts_util:Generating audio for chunk 2_336bdcf2, attempt 1
INFO:tts_util:Generating audio for chunk 3_7d0dcd67, attempt 1


=== STEP 1: Creating Audio Chunks ===
✓ Created 180 audio chunks for job 68e0c9e2356de3c49a902ead
Processed 180 chunks:
  Chunk 1: ID=0f6671d4, Length=32 chars
  Chunk 2: ID=ee238105, Length=738 chars
  Chunk 3: ID=336bdcf2, Length=367 chars
  Chunk 4: ID=7d0dcd67, Length=267 chars
  Chunk 5: ID=2332ad90, Length=334 chars
  Chunk 6: ID=9601774f, Length=260 chars
  Chunk 7: ID=dc5e7df0, Length=102 chars
  Chunk 8: ID=dc0d8d8b, Length=343 chars
  Chunk 9: ID=a81a04c2, Length=155 chars
  Chunk 10: ID=f01f58d3, Length=72 chars
  Chunk 11: ID=643e98c2, Length=351 chars
  Chunk 12: ID=25ba8a44, Length=457 chars
  Chunk 13: ID=ad74e8b7, Length=158 chars
  Chunk 14: ID=ddfa7aa3, Length=434 chars
  Chunk 15: ID=7727600e, Length=214 chars
  Chunk 16: ID=fc5acb51, Length=207 chars
  Chunk 17: ID=4cc146ef, Length=219 chars
  Chunk 18: ID=2ed41938, Length=130 chars
  Chunk 19: ID=01f0b234, Length=386 chars
  Chunk 20: ID=c42b9eb0, Length=28 chars
  Chunk 21: ID=1f01ceb4, Length=27 chars
  Chunk 22:

INFO:tts_util:Successfully generated audio: ./audio_outputs/audio_chunks/20251004_The_Adventure_of_the_Hollow_Cane/000_0f6671d4.wav
INFO:tts_util:Generating audio for chunk 5_9601774f, attempt 1


✓ Processed chunk 0: ./audio_outputs/audio_chunks/20251004_The_Adventure_of_the_Hollow_Cane/000_0f6671d4.wav


INFO:tts_util:Successfully generated audio: ./audio_outputs/audio_chunks/20251004_The_Adventure_of_the_Hollow_Cane/003_7d0dcd67.wav
INFO:tts_util:Generating audio for chunk 6_dc5e7df0, attempt 1


...

✓ Processed chunk 178: ./audio_outputs/audio_chunks/20251004_The_Adventure_of_the_Hollow_Cane/178_30840549.wav


INFO:tts_util:Successfully generated audio: ./audio_outputs/audio_chunks/20251004_The_Adventure_of_the_Hollow_Cane/173_f741b014.wav


✓ Processed chunk 173: ./audio_outputs/audio_chunks/20251004_The_Adventure_of_the_Hollow_Cane/173_f741b014.wav


INFO:tts_util:Successfully generated audio: ./audio_outputs/audio_chunks/20251004_The_Adventure_of_the_Hollow_Cane/179_5af7e125.wav


✓ Processed chunk 179: ./audio_outputs/audio_chunks/20251004_The_Adventure_of_the_Hollow_Cane/179_5af7e125.wav

✓ Audio generation completed!
  Successfully processed: 180
  Failed: 0
  Previously completed: 0
  Total chunks: 180

✓ Audio pipeline completed!
  Audio files saved to: ./audio_outputs/audio_chunks/20251004_The_Adventure_of_the_Hollow_Cane
  Total processed chunks: 180

=== STEP 3: Stitching Audio Chunks ===
Stitching 180 audio chunks for job 68e0c9e2356de3c49a902ead
Gap between chunks: 0.1 seconds
Running FFmpeg to stitch audio files...

✓ Audio stitching completed!
  Processed chunks: 180/180
  Final audio file size: 130.2 MB
  Final audio saved to: ./audio_outputs/final_audio/20251004_The_Adventure_of_the_Hollow_Cane.wav

🎉 Complete audio generation pipeline finished!
  Final audio file: ./audio_outputs/final_audio/20251004_The_Adventure_of_the_Hollow_Cane.wav
